In [9]:
from graphtoolbox import OgbDataLoader, GraphHelper
from ogb.graphproppred import GraphPropPredDataset
import numpy as np
from tqdm import tqdm

In [10]:
dataset_name = 'ogbg-molbbbp'

In [11]:
ogb_data_loader = OgbDataLoader()
X_raw_graphs, y, split_idx = ogb_data_loader.get_processed_dataset(dataset_name)

Downloaded 0.00 GB: 100%|██████████| 2/2 [00:00<00:00,  4.04it/s]


Extracting dataset/bbbp.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 2039/2039 [00:00<00:00, 64067.08it/s]


Saving...


100%|██████████| 2039/2039 [00:00<00:00, 3882.06it/s]


In [12]:
#graph_helper = GraphHelper()

#X_graphs = [graph_helper.edges_to_nodes(graph) for graph in tqdm(X_raw_graphs)]

In [13]:
from graphtoolbox import RandomWalkEmbedder

model = RandomWalkEmbedder()

documents = model.generate_random_walks(X_raw_graphs)

100%|██████████| 2039/2039 [00:11<00:00, 170.83it/s]


In [14]:
text_file = open("documents.txt", "w")
for i, list_of_lists in documents:
    for line in list_of_lists:
        text_file.write(str(i) + ' ' + line + "\n")
text_file.close()

In [17]:
from karateclub import Graph2Vec

model = Graph2Vec(attributed=True)
model.fit(X_raw_graphs)

In [20]:
X = model.get_embedding()

In [22]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

reg = RandomForestClassifier()
reg.fit(X, y)
y_test_predictions = reg.predict(X)

/tmp/ipykernel_522/4178127723.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  reg.fit(X, y)


In [23]:
from ogb.graphproppred import Evaluator

evaluator = Evaluator(name = dataset_name)
input_dict = {"y_true": y.reshape(-1, 1), "y_pred": y_test_predictions.reshape(-1, 1)}
result_dict = evaluator.eval(input_dict)
result_dict

AttributeError: 'list' object has no attribute 'reshape'